# Capstone Project - The Battle of Neighborhoods

## Clustering the Neighborhoods of Amsterdam, The Netherlands

##### Use the Foursquare API to explore  the neighborhoods of Amsterdam and obtain the most common venue categories in each neighborhood. Cluster the neighborhoods using the k-means algorithm. Finally use the Folium library to visualize these clusters superimposed onto the map of Amsterdam. 

##### Obtain the geodata of the neighborhoods of Amsterdam from https://maps.amsterdam.nl/open_geodata/?LANG=en. As there are 481 neighborhoods in Amsterdam use the 99 aggregated neighborhoods, called quarters. Use the data excluding the major waters. For simplicity refer to these quarters as neighborhoods. 

Import all the libraries that will be needed.

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import numpy as np
import requests

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

#!conda install -c conda-forge folium=0.5.0 --yes
import folium 
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

### Import and Explore Dataset

The data is available in different formats. Note: the separator in the **csv** file  is not a comma (,) but a semi-colon (;).  

In [2]:
amsterdam_data=pd.read_csv('GEBIED_BUURTCOMBINATIES_EXWATER.csv',sep=';')
print('There are {} neighborhoods in Amsterdam.'.format(amsterdam_data.shape[0]))
amsterdam_data.head(2)

There are 99 neighborhoods in Amsterdam.


,OBJECTNUMMER,Buurtcombinatie_code,Buurtcombinatie,Stadsdeel_code,Opp_m2,WKT_LNG_LAT,WKT_LAT_LNG,LNG,LAT,Unnamed: 9
0,1,T92,Amstel III/Bullewijk,T,5273610,"POLYGON((4.971842 52.284355,4.970005 52.286362...","POLYGON((52.284355 4.971842,52.286362 4.970005...","4,9505918","52,298741",NaN
1,2,T93,"Bijlmer Centrum (D,F,H)",T,3158100,"POLYGON((4.971421 52.307989,4.965179 52.314811...","POLYGON((52.307989 4.971421,52.314811 4.965179...","4,9545269","52,3153044",NaN


Remove the columns that are not needed. If applicable, translate the remaining column names from Dutch to English. 

In [3]:
amsterdam_data=amsterdam_data.drop(columns=['OBJECTNUMMER','Buurtcombinatie_code','Stadsdeel_code','Opp_m2','WKT_LNG_LAT','WKT_LAT_LNG','Unnamed: 9'])
amsterdam_data = amsterdam_data.rename(columns={'Buurtcombinatie': 'Neighborhood'})
amsterdam_data.head()

,Neighborhood,LNG,LAT
0,Amstel III/Bullewijk,"4,9505918","52,298741"
1,"Bijlmer Centrum (D,F,H)","4,9545269","52,3153044"
2,Gein,"4,99393105","52,29550995"
3,Kinkerbuurt,"4,86544515","52,36808265"
4,Overtoomse Sluis,"4,8598361","52,3597247"


In [4]:
amsterdam_data.dtypes

Neighborhood    object
LNG             object
LAT             object
dtype: object

Fix the  problems with the latitude (LAT) and longitude (LNG) columns: change the decimal separator from comma to dot and change the data type to float.

In [6]:
amsterdam_data["LAT"] = amsterdam_data["LAT"].str.replace(",",".").astype(float)
amsterdam_data["LNG"] = amsterdam_data["LNG"].str.replace(",",".").astype(float)

amsterdam_data.dtypes

Neighborhood     object
LNG             float64
LAT             float64
dtype: object

In [7]:
amsterdam_data.head()

,Neighborhood,LNG,LAT
0,Amstel III/Bullewijk,4.950592,52.298741
1,"Bijlmer Centrum (D,F,H)",4.954527,52.315304
2,Gein,4.993931,52.295510
3,Kinkerbuurt,4.865445,52.368083
4,Overtoomse Sluis,4.859836,52.359725


#### Visualize the neighborhoods 

Use the  **geopy** library to get the latitude and longitude values of Amsterdam.

In [8]:
address = 'Amsterdam'

geolocator = Nominatim(user_agent="ams_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Amsterdam are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Amsterdam are 52.3745403, 4.89797550561798.


Visualize the neighborhoods superimposed on the map of Amsterdam.

In [9]:
# create map of Amsterdam using latitude and longitude values
map_amsterdam = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(amsterdam_data['LAT'], amsterdam_data['LNG'], amsterdam_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_amsterdam)  
    
map_amsterdam

### Define Foursquare Credentials and Version

In [10]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = 'QQ4051V3ODK2UT1EWPLLH04DICITA4RDTUA1ANQKLQ3QRXAP' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

Define the function **getNearbyVenues** which will be used to explore the venues in the neighborhoods of Amsterdam  

In [11]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
       
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Call the above defined function to explore all the neighborhoods of Amsterdam.

In [13]:
LIMIT=100
amsterdam_venues = getNearbyVenues(names=amsterdam_data['Neighborhood'],
                                   latitudes=amsterdam_data['LAT'],
                                   longitudes=amsterdam_data['LNG']
                                  )


Amstel III/Bullewijk
Bijlmer Centrum (D,F,H)
Gein
Kinkerbuurt
Overtoomse Sluis
Erasmuspark
Hoofdweg e.o.
De Weteringschans
Oostelijke Eilanden/Kadijken
Volewijck
Tuindorp Nieuwendam
Tuindorp Buiksloot
Kadoelen
Elzenhagen
Burgwallen-Oude Zijde
Bijlmer Oost (E,G,K)
Driemond
Haarlemmerbuurt
Osdorp-Oost
De Punt
Westelijk Havengebied
Centrale Markt
Westlandgracht
Zuidas
Slotermeer-Noordoost
Geuzenveld
IJburg West
Betondorp
Transvaalbuurt
Indische Buurt Oost
Nieuwe Pijp
Rijnbuurt
Da Costabuurt
Helmersbuurt
Staatsliedenbuurt
Geuzenbuurt
Sloterdijk
Burgwallen-Nieuwe Zijde
Westindische Buurt
Houthavens
Eendracht
Slotermeer-Zuidwest
Osdorp-Midden
Slotervaart Zuid
Oude Pijp
Schinkelbuurt
Apollobuurt
IJselbuurt
Buitenveldert-West
Oosterparkbuurt
Indische Buurt West
Zeeburgereiland/Nieuwe Diep
Nieuwendammerdijk/Buiksloterdijk
Tuindorp Oostzaan
Nellestein
Frederik Hendrikbuurt
Grachtengordel-West
Spaarndammer- en Zeeheldenbuurt
Van Lennepbuurt
Nieuwmarkt/Lastage
De Kolenkit
Slotervaart Noord
Lutkeme

In [14]:
print('Total venues found:', amsterdam_venues.shape[0])
amsterdam_venues.head()

Total venues found: 6557


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Amstel III/Bullewijk,52.298741,4.950592,Hotel Lowell,52.304526,4.950671,Hotel
1,Amstel III/Bullewijk,52.298741,4.950592,De Proefzaak,52.303585,4.950890,Café
2,Amstel III/Bullewijk,52.298741,4.950592,Fitness365,52.298421,4.945692,Gym
3,Amstel III/Bullewijk,52.298741,4.950592,Museum Vrolik,52.293968,4.960014,Science Museum
4,Amstel III/Bullewijk,52.298741,4.950592,Joy,52.305268,4.950310,Hotel


List the number of venues found for each neighborhood.

In [15]:
amsterdam_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Amstel III/Bullewijk,46,46,46,46,46,46
Apollobuurt,100,100,100,100,100,100
Banne Buiksloot,19,19,19,19,19,19
Bedrijventerrein Sloterdijk,9,9,9,9,9,9
Betondorp,29,29,29,29,29,29
"Bijlmer Centrum (D,F,H)",95,95,95,95,95,95
"Bijlmer Oost (E,G,K)",17,17,17,17,17,17
Buikslotermeer,37,37,37,37,37,37
Buitenveldert-Oost,77,77,77,77,77,77


Remove all neighborhoods that have less then 10 venues found in them. 

In [16]:
df=amsterdam_venues.groupby('Neighborhood').count()<10
df=df[df['Venue']==True]
neigh=df.index.values
neigh

array(['Bedrijventerrein Sloterdijk', 'Eendracht', 'Waterland',
       'Westelijk Havengebied'], dtype=object)

In [17]:
for n in neigh:
    print(n)
    amsterdam_venues=amsterdam_venues.drop(amsterdam_venues[amsterdam_venues['Neighborhood']== n].index)

amsterdam_venues.groupby('Neighborhood').count()


Bedrijventerrein Sloterdijk
Eendracht
Waterland
Westelijk Havengebied


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Amstel III/Bullewijk,46,46,46,46,46,46
Apollobuurt,100,100,100,100,100,100
Banne Buiksloot,19,19,19,19,19,19
Betondorp,29,29,29,29,29,29
"Bijlmer Centrum (D,F,H)",95,95,95,95,95,95
"Bijlmer Oost (E,G,K)",17,17,17,17,17,17
Buikslotermeer,37,37,37,37,37,37
Buitenveldert-Oost,77,77,77,77,77,77
Buitenveldert-West,21,21,21,21,21,21


In [18]:
print('There are {} uniques categories.'.format(len(amsterdam_venues['Venue Category'].unique())))

There are 315 uniques categories.


Transform the amsterdam_venues dataframe to make it suitable for the cluster analysis. The dataframe is expended with as many columns as unique venue categories.   

In [19]:
# one hot encoding
amsterdam_onehot = pd.get_dummies(amsterdam_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
amsterdam_onehot['Neighborhood'] = amsterdam_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [amsterdam_onehot.columns[-1]] + list(amsterdam_onehot.columns[:-1])
amsterdam_onehot = amsterdam_onehot[fixed_columns]

amsterdam_onehot.head()

,Zoo Exhibit,Accessories Store,Adult Boutique,Advertising Agency,Afghan Restaurant,African Restaurant,American Restaurant,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brasserie,Breakfast Spot,Brewery,Bridal Shop,Bridge,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Camera Store,Campground,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,Comedy Club,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cruise,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Doner Restaurant,Drugstore,Dumpling Restaurant,Dutch Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Friterie,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Field,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,Nightclub,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Paper / Office Supplies Store,Park,Parking,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Pie Shop,Pier,Pilates Studio,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Print Shop,Pub,Public Art,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Climbing Spot,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shoe Repair,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Ski Area,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stables,Stadium,Steakhouse,

Group the dataframe by neighborhood

In [20]:
amsterdam_grouped = amsterdam_onehot.groupby('Neighborhood').mean().reset_index()
amsterdam_grouped.head()

,Neighborhood,Zoo Exhibit,Accessories Store,Adult Boutique,Advertising Agency,Afghan Restaurant,African Restaurant,American Restaurant,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brasserie,Breakfast Spot,Brewery,Bridal Shop,Bridge,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Camera Store,Campground,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,Comedy Club,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cruise,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Doner Restaurant,Drugstore,Dumpling Restaurant,Dutch Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Friterie,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Field,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,Nature Preserve,Nightclub,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Paper / Office Supplies Store,Park,Parking,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Pie Shop,Pier,Pilates Studio,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Print Shop,Pub,Public Art,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Climbing Spot,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shoe Repair,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Ski Area,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stables,Stadium,Steakhouse,

##### Before running the clustering algorithm display the 10 most common venues in each neighborhood.

In [21]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [22]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = amsterdam_grouped['Neighborhood']

for ind in np.arange(amsterdam_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(amsterdam_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Amstel III/Bullewijk,Hotel,Coffee Shop,Restaurant,Fast Food Restaurant,Food Court,Café,Furniture / Home Store,Cafeteria,Brewery,Convenience Store
1,Apollobuurt,Hotel,Restaurant,Bakery,French Restaurant,Plaza,Coffee Shop,Italian Restaurant,Bistro,Japanese Restaurant,Bar
2,Banne Buiksloot,Bus Stop,Soccer Field,Tennis Court,Grocery Store,Farm,Sports Club,Stables,Supermarket,Fast Food Restaurant,Park
3,Betondorp,Soccer Field,Café,Stadium,Scandinavian Restaurant,Supermarket,Sports Bar,Flower Shop,Fast Food Restaurant,Museum,Bus Stop
4,"Bijlmer Centrum (D,F,H)",Coffee Shop,Office,Chinese Restaurant,Restaurant,Park,Bakery,Hotel,Food Court,Plaza,Supermarket
5,"Bijlmer Oost (E,G,K)",Supermarket,Bar,Bakery,Metro Station,Bus Stop,Discount Store,Park,Turkish Restaurant,South American Restaurant,Shopping Mall
6,Buikslotermeer,Bus Stop,Supermarket,Park,Clothing Store,Soccer Field,Shopping Mall,Market,Sporting Goods Shop,Martial Arts Dojo,Flea Market
7,Buitenveldert-Oost,Hotel,Korean Restaurant,Gym,Gym / Fitness Center,Sandwich Place,Coffee Shop,Playground,Bakery,Supermarket,Park
8,Buitenveldert-West,Restaurant,Harbor / Marina,Tennis Court,Flower Shop,Spa,Garden,Dog Run,Gastropub,Stables,Lake
9,Burgwallen-Nieuwe Zijde,Hotel,Bar,Marijuana Dispensary,Café,Cocktail Bar,French Restaurant,Restaurant,Beer Bar,Sandwich Place,Pub


In [23]:
# set number of clusters
kclusters = 20

#remove the column containing the name of the Neighborhood
amsterdam_grouped_clustering = amsterdam_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(amsterdam_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_ 

array([18, 10,  3, 12, 18, 16,  6, 18,  9, 19, 19,  2,  2,  2, 10,  0,  1,
       10,  5,  6,  0,  0,  2,  7,  2,  1, 19, 10,  2,  2,  4,  0,  2,  0,
       15, 17, 15, 13, 10,  0, 10, 19,  3,  2,  0,  8, 14, 12,  0, 11, 10,
        6, 19, 13,  0,  0,  0, 10, 10,  6,  1,  1, 10,  2,  0, 18, 18, 18,
        0, 18,  0,  1,  1, 18, 18,  0,  2,  0, 10,  6,  6,  6,  0,  2, 13,
       10,  6, 10, 10,  2, 18, 10, 17, 10, 18], dtype=int32)

Insert the cluster label column into the dataframe and the geocoordinates

In [24]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

amsterdam_merged = amsterdam_data
# merge amsterdam_grouped with amsterdam_data to add latitude/longitude for each neighborhood
amsterdam_merged = amsterdam_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood',how='right')

amsterdam_merged.shape

(95, 14)

##### Visualize the map of Amsterdam with the neighborhoods superimposed, colorcoded as a function of the cluster they belong to.

In [27]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.gist_rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(amsterdam_merged['LAT'], amsterdam_merged['LNG'], amsterdam_merged['Neighborhood'], amsterdam_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Conclusions

##### Display the top 10 venues for each neighborhood for each cluster.

In [28]:
amsterdam_merged.loc[amsterdam_merged['Cluster Labels'] == 0, amsterdam_merged.columns[[0] + list(range(4, amsterdam_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Erasmuspark,Coffee Shop,Supermarket,Restaurant,Bar,Café,Fast Food Restaurant,Bakery,Fish Market,Food & Drink Shop,Deli / Bodega
29,Indische Buurt Oost,Restaurant,Café,Coffee Shop,Supermarket,Bakery,Playground,Bar,Harbor / Marina,Cocktail Bar,Modern European Restaurant
36,Sloterdijk,Restaurant,Hotel,Music Venue,Farm,Bistro,Supermarket,Café,Mediterranean Restaurant,Chinese Restaurant,Casino
39,Houthavens,Restaurant,Coffee Shop,Indie Movie Theater,Italian Restaurant,Café,Pizza Place,French Restaurant,Snack Place,Theater,Brewery
45,Schinkelbuurt,Restaurant,Bar,Café,Coffee Shop,Hotel,Gym / Fitness Center,Athletics & Sports,Bakery,Ice Cream Shop,Pizza Place
57,Spaarndammer- en Zeeheldenbuurt,Restaurant,Coffee Shop,Café,Music Venue,Theater,Indie Movie Theater,Snack Place,Italian Restaurant,Pizza Place,Farm
60,De Kolenkit,Turkish Restaurant,Park,Restaurant,Hotel,Fast Food Restaurant,Café,Coffee Shop,Bakery,Hostel,Vegetarian / Vegan Restaurant
68,Hoofddorppleinbuurt,Restaurant,Coffee Shop,Café,Bar,Bakery,Gym / Fitness Center,Athletics & Sports,Hotel,Snack Place,Gym
75,Landlust,Café,Park,Restaurant,Coffee Shop,Bakery,Sandwich Place,Hostel,Shopping Mall,Seafood Restaurant,Scandinavian Restaurant
77,Stadionbuurt,Restaurant,Bakery,Café,Bar,Coffee Shop,Snack Place,Athletics & Sports,Italian Restaurant,Bagel Shop,Ice Cream Shop


In [29]:
amsterdam_merged.loc[amsterdam_merged['Cluster Labels'] == 1, amsterdam_merged.columns[[0] + list(range(4, amsterdam_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Osdorp-Oost,Turkish Restaurant,Supermarket,Snack Place,Clothing Store,Tram Station,Shopping Mall,Chinese Restaurant,Pharmacy,Fast Food Restaurant,Indonesian Restaurant
19,De Punt,Supermarket,Fast Food Restaurant,Tram Station,Bakery,Turkish Restaurant,Clothing Store,Hotel,Garden Center,Plaza,Sporting Goods Shop
24,Slotermeer-Noordoost,Hotel,Turkish Restaurant,Park,Dessert Shop,Other Repair Shop,Gym,Supermarket,Sandwich Place,Discount Store,Fast Food Restaurant
25,Geuzenveld,Bus Stop,Supermarket,Park,Tram Station,Italian Restaurant,Furniture / Home Store,Fast Food Restaurant,Athletics & Sports,Greek Restaurant,Arts & Entertainment
41,Slotermeer-Zuidwest,Turkish Restaurant,Tram Station,Park,Dessert Shop,Café,Plaza,Discount Store,Bus Stop,Museum,Fast Food Restaurant
42,Osdorp-Midden,Supermarket,Tram Station,Snack Place,Clothing Store,Turkish Restaurant,Bakery,Gym / Fitness Center,Hotel,Drugstore,Shopping Mall


In [30]:
amsterdam_merged.loc[amsterdam_merged['Cluster Labels'] == 2, amsterdam_merged.columns[[0] + list(range(4, amsterdam_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Kinkerbuurt,Bar,Italian Restaurant,Coffee Shop,Restaurant,Yoga Studio,Grocery Store,Vegetarian / Vegan Restaurant,Café,Pizza Place,Burger Joint
4,Overtoomse Sluis,Restaurant,Bar,Café,Bistro,Coffee Shop,Ethiopian Restaurant,Yoga Studio,Music Venue,BBQ Joint,Marijuana Dispensary
6,Hoofdweg e.o.,Bar,Restaurant,Italian Restaurant,Coffee Shop,Pizza Place,Deli / Bodega,Bistro,Yoga Studio,Café,Gym / Fitness Center
17,Haarlemmerbuurt,Bar,Restaurant,Marijuana Dispensary,Italian Restaurant,Café,Bakery,Sandwich Place,Bistro,Coffee Shop,Deli / Bodega
21,Centrale Markt,Italian Restaurant,Coffee Shop,Bar,Café,Restaurant,Ice Cream Shop,Indonesian Restaurant,Pizza Place,Snack Place,Grocery Store
32,Da Costabuurt,Bar,Coffee Shop,Italian Restaurant,Pizza Place,Café,Restaurant,Yoga Studio,Cheese Shop,Clothing Store,Cocktail Bar
33,Helmersbuurt,Bar,Café,Coffee Shop,Hotel,Italian Restaurant,Hostel,Restaurant,Ethiopian Restaurant,Bagel Shop,Dutch Restaurant
34,Staatsliedenbuurt,Bar,Italian Restaurant,Coffee Shop,Café,Thai Restaurant,Bistro,Sandwich Place,Pizza Place,Restaurant,Plaza
35,Geuzenbuurt,Bar,Italian Restaurant,Coffee Shop,Restaurant,Café,Pizza Place,Hotel,Greek Restaurant,Grocery Store,Yoga Studio
38,Westindische Buurt,Restaurant,Bar,Ethiopian Restaurant,Café,Vegetarian / Vegan Restaurant,Bistro,Breakfast Spot,Coffee Shop,Park,Music Venue


In [31]:
amsterdam_merged.loc[amsterdam_merged['Cluster Labels'] == 3, amsterdam_merged.columns[[0] + list(range(4, amsterdam_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Kadoelen,Bus Stop,Soccer Field,Tennis Court,Park,Shopping Mall,Sports Club,Stables,Supermarket,Ski Area,Gas Station
70,Banne Buiksloot,Bus Stop,Soccer Field,Tennis Court,Grocery Store,Farm,Sports Club,Stables,Supermarket,Fast Food Restaurant,Park


In [32]:
amsterdam_merged.loc[amsterdam_merged['Cluster Labels'] == 4, amsterdam_merged.columns[[0] + list(range(4, amsterdam_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
72,Holendrecht/Reigersbos,Pizza Place,Supermarket,Chinese Restaurant,Shopping Mall,Grocery Store,Train Station,Snack Place,Bus Station,Beer Garden,Science Museum


In [33]:
amsterdam_merged.loc[amsterdam_merged['Cluster Labels'] == 5, amsterdam_merged.columns[[0] + list(range(4, amsterdam_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,Driemond,Restaurant,Athletics & Sports,Plaza,Monument / Landmark,Convention Center,Hotel,Print Shop,Soccer Field,Hotel Bar,Arcade


In [34]:
amsterdam_merged.loc[amsterdam_merged['Cluster Labels'] == 6, amsterdam_merged.columns[[0] + list(range(4, amsterdam_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Tuindorp Nieuwendam,Bus Stop,Supermarket,Park,Harbor / Marina,Snack Place,Indonesian Restaurant,Electronics Store,Drugstore,Bar,Liquor Store
11,Tuindorp Buiksloot,Bus Stop,Park,Clothing Store,Bakery,Supermarket,Harbor / Marina,Hotel,Department Store,Electronics Store,Chinese Restaurant
13,Elzenhagen,Bus Stop,Supermarket,Clothing Store,Electronics Store,Soccer Field,Fast Food Restaurant,Breakfast Spot,Convenience Store,Flea Market,Metro Station
52,Nieuwendammerdijk/Buiksloterdijk,Bus Stop,Clothing Store,Supermarket,Bakery,Electronics Store,Sporting Goods Shop,Flea Market,Soccer Field,Liquor Store,Metro Station
53,Tuindorp Oostzaan,Bus Stop,Soccer Field,Chinese Restaurant,Shopping Mall,Shop & Service,Flea Market,Sandwich Place,Bus Station,Steakhouse,Supermarket
67,Oostzanerwerf,Bus Stop,Supermarket,Soccer Field,Flower Shop,Park,Shop & Service,Dutch Restaurant,BBQ Joint,Grocery Store,Bakery
69,Waterlandpleinbuurt,Bus Stop,Athletics & Sports,Supermarket,Plaza,Sandwich Place,Seafood Restaurant,Park,Martial Arts Dojo,Grocery Store,Discount Store
80,Buikslotermeer,Bus Stop,Supermarket,Park,Clothing Store,Soccer Field,Shopping Mall,Market,Sporting Goods Shop,Martial Arts Dojo,Flea Market


In [35]:
amsterdam_merged.loc[amsterdam_merged['Cluster Labels'] == 7, amsterdam_merged.columns[[0] + list(range(4, amsterdam_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Gein,Lake,Playground,Gym,Smoke Shop,Metro Station,Supermarket,Bakery,Snack Place,IT Services,Event Space


In [36]:
amsterdam_merged.loc[amsterdam_merged['Cluster Labels'] == 8, amsterdam_merged.columns[[0] + list(range(4, amsterdam_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
62,Lutkemeer/Ookmeer,Clothing Store,Snack Place,Toy / Game Store,Fast Food Restaurant,Sandwich Place,Pet Store,BBQ Joint,Discount Store,Turkish Restaurant,Sporting Goods Shop


In [37]:
amsterdam_merged.loc[amsterdam_merged['Cluster Labels'] == 9, amsterdam_merged.columns[[0] + list(range(4, amsterdam_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,Buitenveldert-West,Restaurant,Harbor / Marina,Tennis Court,Flower Shop,Spa,Garden,Dog Run,Gastropub,Stables,Lake


In [38]:
amsterdam_merged.loc[amsterdam_merged['Cluster Labels'] == 10, amsterdam_merged.columns[[0] + list(range(4, amsterdam_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,De Weteringschans,Café,Coffee Shop,Art Museum,Steakhouse,Clothing Store,Hotel,French Restaurant,Burger Joint,Restaurant,Sandwich Place
8,Oostelijke Eilanden/Kadijken,Zoo Exhibit,Restaurant,Bar,Café,Museum,Hotel,Gastropub,Breakfast Spot,Pizza Place,Performing Arts Venue
28,Transvaalbuurt,Café,Coffee Shop,Hotel,French Restaurant,Italian Restaurant,Restaurant,Bar,Turkish Restaurant,Breakfast Spot,Ice Cream Shop
30,Nieuwe Pijp,Italian Restaurant,Japanese Restaurant,Pizza Place,Seafood Restaurant,Bar,Breakfast Spot,Salad Place,French Restaurant,Yoga Studio,Bakery
44,Oude Pijp,Italian Restaurant,Pizza Place,Hotel,Bar,Seafood Restaurant,French Restaurant,Breakfast Spot,Deli / Bodega,Salad Place,Vegetarian / Vegan Restaurant
46,Apollobuurt,Hotel,Restaurant,Bakery,French Restaurant,Plaza,Coffee Shop,Italian Restaurant,Bistro,Japanese Restaurant,Bar
47,IJselbuurt,Italian Restaurant,Ice Cream Shop,Café,Restaurant,Hotel,Pizza Place,Breakfast Spot,Yoga Studio,Japanese Restaurant,French Restaurant
49,Oosterparkbuurt,Hotel,Restaurant,Italian Restaurant,Café,Coffee Shop,Bar,Breakfast Spot,French Restaurant,Zoo Exhibit,Bakery
50,Indische Buurt West,Restaurant,Bar,Café,Coffee Shop,Turkish Restaurant,Vegetarian / Vegan Restaurant,Italian Restaurant,Gym / Fitness Center,Market,Bakery
65,Dapperbuurt,Bar,Coffee Shop,Café,Turkish Restaurant,Italian Restaurant,French Restaurant,Vegetarian / Vegan Restaurant,Market,Ice Cream Shop,Thai Restaurant


In [39]:
amsterdam_merged.loc[amsterdam_merged['Cluster Labels'] == 11, amsterdam_merged.columns[[0] + list(range(4, amsterdam_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
54,Nellestein,Restaurant,Lake,Playground,Hotel,Park,Stables,Bus Station,Campground,Monument / Landmark,Convention Center


In [40]:
amsterdam_merged.loc[amsterdam_merged['Cluster Labels'] == 12, amsterdam_merged.columns[[0] + list(range(4, amsterdam_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,Betondorp,Soccer Field,Café,Stadium,Scandinavian Restaurant,Supermarket,Sports Bar,Flower Shop,Fast Food Restaurant,Museum,Bus Stop
92,Middenmeer,Café,Soccer Field,Fast Food Restaurant,Coffee Shop,Bus Stop,Train Station,Restaurant,Playground,Stadium,Skating Rink


In [41]:
amsterdam_merged.loc[amsterdam_merged['Cluster Labels'] == 13, amsterdam_merged.columns[[0] + list(range(4, amsterdam_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Volewijck,Café,Furniture / Home Store,Bakery,Park,Sporting Goods Shop,Market,Bus Stop,Restaurant,Turkish Restaurant,Arcade
90,IJplein/Vogelbuurt,Café,Bus Stop,Park,Turkish Restaurant,Brewery,Restaurant,Market,Clothing Store,Bakery,Supermarket
95,Noordelijke IJ-oevers Oost,Bus Stop,Café,Music Venue,Bar,Restaurant,Harbor / Marina,Furniture / Home Store,Park,Bakery,Camera Store


In [42]:
amsterdam_merged.loc[amsterdam_merged['Cluster Labels'] == 14, amsterdam_merged.columns[[0] + list(range(4, amsterdam_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
74,Middelveldsche Akerploder,Tram Station,Trail,Hotel,Museum,Music Store,Spa,Drugstore,Bed & Breakfast,Supermarket,Garden Center


In [69]:
amsterdam_merged.loc[amsterdam_merged['Cluster Labels'] == 15, amsterdam_merged.columns[[0] + list(range(4, amsterdam_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
62,Lutkemeer/Ookmeer,Clothing Store,Discount Store,Turkish Restaurant,Café,Fast Food Restaurant,Electronics Store,Sandwich Place,Pet Store,Sporting Goods Shop,Piano Bar


In [43]:
amsterdam_merged.loc[amsterdam_merged['Cluster Labels'] == 16, amsterdam_merged.columns[[0] + list(range(4, amsterdam_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,"Bijlmer Oost (E,G,K)",Supermarket,Bar,Bakery,Metro Station,Bus Stop,Discount Store,Park,Turkish Restaurant,South American Restaurant,Shopping Mall


In [44]:
amsterdam_merged.loc[amsterdam_merged['Cluster Labels'] == 17, amsterdam_merged.columns[[0] + list(range(4, amsterdam_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
26,IJburg West,Farmers Market,Italian Restaurant,Tram Station,Park,Shopping Mall,Asian Restaurant,Supermarket,Beach,Discount Store,Fast Food Restaurant
51,Zeeburgereiland/Nieuwe Diep,Farmers Market,Harbor / Marina,Hotel,Beach,Supermarket,Bookstore,Soccer Field,Fish Market,Fast Food Restaurant,Shopping Mall


In [45]:
amsterdam_merged.loc[amsterdam_merged['Cluster Labels'] == 18, amsterdam_merged.columns[[0] + list(range(4, amsterdam_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Amstel III/Bullewijk,Hotel,Coffee Shop,Restaurant,Fast Food Restaurant,Food Court,Café,Furniture / Home Store,Cafeteria,Brewery,Convenience Store
1,"Bijlmer Centrum (D,F,H)",Coffee Shop,Office,Chinese Restaurant,Restaurant,Park,Bakery,Hotel,Food Court,Plaza,Supermarket
22,Westlandgracht,Supermarket,Coffee Shop,Café,Restaurant,Gym,Hotel,Gym / Fitness Center,Italian Restaurant,Bar,Soccer Field
23,Zuidas,Coffee Shop,Hotel,Supermarket,Restaurant,Gym,Italian Restaurant,Breakfast Spot,Salad Place,Sushi Restaurant,Bakery
31,Rijnbuurt,Café,Italian Restaurant,Supermarket,Restaurant,Gym / Fitness Center,Hotel,Japanese Restaurant,Coffee Shop,Breakfast Spot,Snack Place
43,Slotervaart Zuid,Coffee Shop,Tram Station,Supermarket,Plaza,Chinese Restaurant,Asian Restaurant,Convenience Store,Miscellaneous Shop,Bakery,Construction & Landscaping
61,Slotervaart Noord,Supermarket,Liquor Store,Pharmacy,Gym,Beach,Moroccan Restaurant,Pool,Museum,Metro Station,Furniture / Home Store
63,Prinses Irenebuurt e.o.,Hotel,Italian Restaurant,Restaurant,Plaza,Japanese Restaurant,Salad Place,Breakfast Spot,Coffee Shop,Bar,Bakery
64,Buitenveldert-Oost,Hotel,Korean Restaurant,Gym,Gym / Fitness Center,Sandwich Place,Coffee Shop,Playground,Bakery,Supermarket,Park
82,Scheldebuurt,Japanese Restaurant,Restaurant,Italian Restaurant,Café,Hotel,Ice Cream Shop,Supermarket,Coffee Shop,Breakfast Spot,Snack Place


In [46]:
amsterdam_merged.loc[amsterdam_merged['Cluster Labels'] == 19, amsterdam_merged.columns[[0] + list(range(4, amsterdam_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Burgwallen-Oude Zijde,Hotel,Bar,French Restaurant,Marijuana Dispensary,Café,Coffee Shop,Thai Restaurant,Bakery,Cheese Shop,Chinese Restaurant
37,Burgwallen-Nieuwe Zijde,Hotel,Bar,Marijuana Dispensary,Café,Cocktail Bar,French Restaurant,Restaurant,Beer Bar,Sandwich Place,Pub
56,Grachtengordel-West,Hotel,Bar,Bookstore,Café,Sandwich Place,French Restaurant,Cheese Shop,Marijuana Dispensary,Dessert Shop,Ice Cream Shop
59,Nieuwmarkt/Lastage,Bar,Hotel,Pub,Dessert Shop,Coffee Shop,Thai Restaurant,History Museum,Restaurant,Café,Mediterranean Restaurant
87,Jordaan,Hotel,Bar,Restaurant,Pizza Place,Cheese Shop,Café,French Restaurant,Italian Restaurant,Cocktail Bar,Marijuana Dispensary
